In [49]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

def parse_offset_file(filepath):
    """Parses a .dat file with multiple named arrays into a dict."""
    arrays = {}
    with open(filepath, "r") as f:
        lines = f.readlines()

    current_key = None
    current_vals = []
    for line in lines:
        if "=" in line:
            if current_key is not None:
                arrays[current_key] = np.array(current_vals, dtype=float)
            current_key = line.split("=")[0].strip()
            current_vals = []
        else:
            current_vals.extend([float(x) for x in line.strip().split()])
    
    # Add the last array
    if current_key is not None:
        arrays[current_key] = np.array(current_vals, dtype=float)

    return arrays

def plot_residuals_to_pdf(file1, file2, output_pdf):
    data1 = parse_offset_file(file1)
    data2 = parse_offset_file(file2)

    with PdfPages(output_pdf) as pdf:
        for key in data1:
            if key in data2:
                if data1[key].shape != data2[key].shape:
                    print(f"Skipping {key}: shape mismatch")
                    continue
                residual = data1[key] - data2[key]

                plt.figure(figsize=(10, 4))
                plt.plot(residual, marker='o', linestyle='-', markersize=3)
                plt.title(f"Residuals for {key}")
                plt.xlabel("Index")
                plt.ylabel("Residual (File1 - File2)")
                plt.grid(True)
                plt.tight_layout()
                pdf.savefig()
                plt.close()
            else:
                print(f"Key {key} not found in second file.")

# Replace with actual file paths
file1 = "GEN3_He3_vimukthi_0514_rm_RF_2cofGEN3_He3_vimukthi_0514_rm_RF_2.dat"
file2 = "GEN3_He3_vimukthi_0514_2cofGEN3_He3_vimukthi_0514_2.dat"
output_pdf = "residuals_output.pdf"

plot_residuals_to_pdf(file1, file2, output_pdf)
